In [47]:
%load_ext autoreload
%autoreload 2

from scripts.wandb_experiments.gr_comparison import GrComparison
import multiprocessing as mp
import torch

torch.set_grad_enabled(False)
mp.set_start_method("spawn", force=True)

e = GrComparison("metal_cat_feature_blend_alpha")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
e.sync_experiment()


Would execute 5 new runs:
- o_0
- o_1
- o_2
- o_3
- o_4

Would delete 0 outdated runs:

Do you want to continue? (y/n)
Deleting 0 runs
Executing 5 runs


wandb: Currently logged in as: romeu. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /home/jorge/thesis/wandb/run-20250414_174601-du3njl9t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run o_0
wandb: ⭐️ View project at https://wandb.ai/romeu/diffusion-3D-features
wandb: 🚀 View run at https://wandb.ai/romeu/diffusion-3D-features/runs/du3njl9t
wandb:   4 of 4 files downloaded.  
100%|██████████| 10/10 [00:27<00:00,  2.78s/it]


Moviepy - Building video /tmp/tmp5gk6w315.mp4.
Moviepy - Writing video /tmp/tmp5gk6w315.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmp5gk6w315.mp4


wandb: Adding directory to artifact (/tmp/local_artifacts/video/video)... Done. 0.0s
wandb:                                                                                
wandb: 
wandb: Run summary:
wandb: hash 67cea92f4e31cd004096...
wandb: 
wandb: 🚀 View run o_0 at: https://wandb.ai/romeu/diffusion-3D-features/runs/du3njl9t
wandb: ⭐️ View project at: https://wandb.ai/romeu/diffusion-3D-features
wandb: Synced 5 W&B file(s), 0 media file(s), 21 artifact file(s) and 1 other file(s)
wandb: Find logs at: ./wandb/run-20250414_174601-du3njl9t/logs
wandb: Currently logged in as: romeu. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /home/jorge/thesis/wandb/run-20250414_174653-x9mg2r7q
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run o_1
wandb: ⭐️ View project at https://wandb.ai/romeu/diffusion-3D-features
wandb: 🚀 View run at https://wandb.ai/romeu/diffusion-3D-features/runs/x9mg2r7q
wandb:   

...

Moviepy - Done !
Moviepy - video ready /tmp/tmp_hwg7cwr.mp4


wandb: Adding directory to artifact (/tmp/local_artifacts/video/video)... Done. 0.0s
wandb:                                                                                
wandb: 
wandb: Run summary:
wandb: hash bcf7614174a5fa601fc8...
wandb: 
wandb: 🚀 View run o_3 at: https://wandb.ai/romeu/diffusion-3D-features/runs/63xqq5d1
wandb: ⭐️ View project at: https://wandb.ai/romeu/diffusion-3D-features
wandb: Synced 5 W&B file(s), 0 media file(s), 21 artifact file(s) and 1 other file(s)
wandb: Find logs at: ./wandb/run-20250414_174846-63xqq5d1/logs
wandb: Currently logged in as: romeu. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /home/jorge/thesis/wandb/run-20250414_174938-29xtmtux
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run o_4
wandb: ⭐️ View project at https://wandb.ai/romeu/diffusion-3D-features
wandb: 🚀 View run at https://wandb.ai/romeu/diffusion-3D-features/runs/29xtmtux
wandb:   

Moviepy - Building video /tmp/tmpdutbdqbm.mp4.
Moviepy - Writing video /tmp/tmpdutbdqbm.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpdutbdqbm.mp4


wandb: Adding directory to artifact (/tmp/local_artifacts/video/video)... Done. 0.0s
wandb:                                                                                
wandb: 
wandb: Run summary:
wandb: hash 450a4269c0f34fb5b603...
wandb: 
wandb: 🚀 View run o_4 at: https://wandb.ai/romeu/diffusion-3D-features/runs/29xtmtux
wandb: ⭐️ View project at: https://wandb.ai/romeu/diffusion-3D-features
wandb: Synced 5 W&B file(s), 0 media file(s), 21 artifact file(s) and 1 other file(s)
wandb: Find logs at: ./wandb/run-20250414_174938-29xtmtux/logs


In [62]:
data = e.get_data()

wandb:   20 of 20 files downloaded.  
wandb:   20 of 20 files downloaded.  
wandb:   20 of 20 files downloaded.  
wandb:   20 of 20 files downloaded.  
wandb:   20 of 20 files downloaded.  


In [63]:
from text3d2video.utilities.ipython_utils import display_vid

vid = e.comparison_vid(data)
display_vid(vid, width=1000)

In [58]:
from omegaconf import OmegaConf

from wandb_util.wandb_util import omegaconf_create_nested


omegaconf_create_nested({'a.b': 1})

{'a': {'b': 1}}